# Walmart Sales Forecasting-- Feature Engineering
Author-@attharva_j

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 6.03 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%load_ext cudf.pandas
import pandas as pd

In [ ]:
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

In [3]:
# import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from tqdm import tqdm
import pickle
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
CV_PART_PRCNT = 10

In [5]:
CV_PART_PRCNT_LIST = [2, 5, 10, 15, 20, 25, 30]

In [8]:
# %%timeit
df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/final_dataframe.pkl")

FileNotFoundError: ignored

In [ ]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,1,2011,no_event,no_event,no_event,no_event,0,0,0,0.46
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,1,2011,no_event,no_event,no_event,no_event,0,0,0,0.46
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,1,2011,no_event,no_event,no_event,no_event,0,0,0,0.46
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,2,2011,no_event,no_event,no_event,no_event,1,1,0,0.46
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,2,2011,no_event,no_event,no_event,no_event,1,0,1,0.46


In [ ]:
df.columns

StringIndex(['id' 'item_id' 'dept_id' 'cat_id' 'store_id' 'state_id' 'd' 'sales'
 'date' 'wm_yr_wk' 'weekday' 'wday' 'month' 'year' 'event_name_1'
 'event_type_1' 'event_name_2' 'event_type_2' 'snap_CA' 'snap_TX'
 'snap_WI' 'sell_price'], dtype='object')

In [ ]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 46027957 entries, 0 to 46027956
Data columns (total 22 columns):
 #   Column        Dtype
---  ------        -----
 0   id            object
 1   item_id       object
 2   dept_id       object
 3   cat_id        object
 4   store_id      object
 5   state_id      object
 6   d             object
 7   sales         int64
 8   date          object
 9   wm_yr_wk      int64
 10  weekday       object
 11  wday          int64
 12  month         int64
 13  year          int64
 14  event_name_1  object
 15  event_type_1  object
 16  event_name_2  object
 17  event_type_2  object
 18  snap_CA       int64
 19  snap_TX       int64
 20  snap_WI       int64
 21  sell_price    float64
dtypes: float64(1), int64(8), object(13)
memory usage: 10.4+ GB


In [ ]:
#Reading up the dataframes
train=df
test=pd.read_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/final_dataframe_test.csv')
final_test=pd.read_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/final_future_data.csv')

### Encode categorcal variables

In [ ]:
# cols=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1',
#       'event_name_2', 'event_type_1', 'event_type_2', 'year']

cols=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1',
      'event_name_2', 'event_type_1', 'event_type_2']

for colname in cols:
  print(f"\rPreprocessing categorical label: {colname}")
  lbl=LabelEncoder()
  train[colname]=lbl.fit_transform(train[colname])
  test[colname]=lbl.transform(test[colname])
  final_test[colname]=lbl.transform(final_test[colname])
  pickle.dump(lbl,open(f'/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/label_encoder_{colname}.sav','wb'))


Preprocessing categorical label: item_id
Preprocessing categorical label: dept_id
Preprocessing categorical label: cat_id
Preprocessing categorical label: store_id
Preprocessing categorical label: state_id
Preprocessing categorical label: event_name_1
Preprocessing categorical label: event_name_2
Preprocessing categorical label: event_type_1
Preprocessing categorical label: event_type_2


In [ ]:
# Converting snap_CA,snap_WI,snap_TX into one feature named snap
# set 'snap' column value to 1 if either of snap_CA, snap_TX, snap_WI is 1 else set as 0
train.loc[train['state_id'] == 'CA', 'snap'] = train.loc[train['state_id'] == 'CA']['snap_CA']
train.loc[train['state_id'] == 'TX', 'snap'] = train.loc[train['state_id'] == 'TX']['snap_TX']
train.loc[train['state_id'] == 'WI', 'snap'] = train.loc[train['state_id'] == 'WI']['snap_WI']
train.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)


test.loc[test['state_id'] == 'CA', 'snap'] = test.loc[test['state_id'] == 'CA']['snap_CA']
test.loc[test['state_id'] == 'TX', 'snap'] = test.loc[test['state_id'] == 'TX']['snap_TX']
test.loc[test['state_id'] == 'WI', 'snap'] = test.loc[test['state_id'] == 'WI']['snap_WI']
test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

final_test.loc[final_test['state_id'] == 'CA', 'snap'] = final_test.loc[final_test['state_id'] == 'CA']['snap_CA']
final_test.loc[final_test['state_id'] == 'TX', 'snap'] = final_test.loc[final_test['state_id'] == 'TX']['snap_TX']
final_test.loc[final_test['state_id'] == 'WI', 'snap'] = final_test.loc[final_test['state_id'] == 'WI']['snap_WI']
final_test.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

In [ ]:
# Weekday as wday are similar features so we remove it
#I will make no use of  wm_yr_wk feature
train.drop('weekday',axis=1,inplace=True)
train.drop('wm_yr_wk',axis=1,inplace=True)

test.drop('weekday',axis=1,inplace=True)
test.drop('wm_yr_wk',axis=1,inplace=True)

final_test.drop('weekday',axis=1,inplace=True)
final_test.drop('wm_yr_wk',axis=1,inplace=True)

### 1.Date Related Features

In [ ]:
#Reference https://stackoverflow.com/questions/2600775/how-to-get-week-number-in-python/32638267#:~:text=()%5B1%5D%2024-,datetime.,for%20the%20given%20date%20instance.&text=You%20can%20get%20the%20week%20number%20directly%20from%20datetime%20as%20string.
def get_week_number(x):
    """This Function is used to get weeknumber of particular date"""
    date=calendar.datetime.date.fromisoformat(x)
    return date.isocalendar()[1]

In [ ]:
train['week_number']=train['date'].apply(lambda x:get_week_number(x))
test['week_number']=test['date'].apply(lambda x:get_week_number(x))
final_test['week_number']=final_test['date'].apply(lambda x:get_week_number(x))

### 2.Season

In [ ]:
#https://www.universaltraveller.com.au/destinations/los-angeles/weather
def get_season(x):
    """This function is used to get season in US according to various months"""
    if x in [12,1,2]:
        return 0      #"Winter"
    elif x in [3,4,5]:
        return 1   #"Spring"
    elif x in [6,7,8]:
        return 2   #"Summer"
    else:
        return 3   #"Autumn"

In [ ]:
train['season']=train['month'].apply(lambda x:get_season(x))
test['season']=test['month'].apply(lambda x:get_season(x))
final_test['season']=final_test['month'].apply(lambda x:get_season(x))

### 3.Quarter Start

In [ ]:
def check_if_quater_begin(x):
    """This is used to check if day is begining of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and (month in [1,4,7,9])) else 0

In [ ]:
train['quater_start']=train['date'].apply(lambda x:check_if_quater_begin(x))
test['quater_start']=test['date'].apply(lambda x:check_if_quater_begin(x))
final_test['quater_start']=final_test['date'].apply(lambda x:check_if_quater_begin(x))

### 4.Quarter End

In [ ]:
#Reference https://www.lawinsider.com/dictionary/quarter-end
def check_if_quater_end(x):
    """This is used to check if day is end of quater"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    if (day==31 and month==3) or (day==30 and month==6) or (day==30 and month==9) or (day==31 and month==12):
        return 1
    else:
        return 0

In [ ]:
train['quater_end']=train['date'].apply(lambda x:check_if_quater_end(x))
test['quater_end']=test['date'].apply(lambda x:check_if_quater_end(x))
final_test['quater_end']=final_test['date'].apply(lambda x:check_if_quater_end(x))

### 5.Month Start

In [ ]:
def month_start(x):
    """This is used to check if day is begining of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    return 1 if day==1 else 0

In [ ]:
train['month_start']=train['date'].apply(lambda x:month_start(x))
test['month_start']=test['date'].apply(lambda x:month_start(x))
final_test['month_start']=final_test['date'].apply(lambda x:month_start(x))

### 6.Month End

In [ ]:
def month_end(x):
    """This is used to check if day is end of month"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    year=calendar.datetime.date.fromisoformat(x).year
    leap_yr=(year%4==0) # to check if it is a leap year
    val=(day==31 and month==1) or (day==29 if leap_yr else day==28) or (day==31 and month==3) or (day==30 and month==4) or\
        (day==31 and month==5) or (day==30 and month==6) or (day==31 and month==7) or (day==31 and month==8) or\
        (day==30 and month==9) or (day==31 and month==10) or (day==30 and month==11) or (day==31 and month==12)
    return 1 if val else 0

In [ ]:
train['month_end']=train['date'].apply(lambda x:month_end(x))
test['month_end']=test['date'].apply(lambda x:month_end(x))
final_test['month_end']=final_test['date'].apply(lambda x:month_end(x))

### 7.Year Start

In [ ]:
def year_start(x):
    """This is used to check if day is begining of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==1 and month==1) else 0

In [ ]:
train['year_start']=train['date'].apply(lambda x:year_start(x))
test['year_start']=test['date'].apply(lambda x:year_start(x))
final_test['year_start']=final_test['date'].apply(lambda x:year_start(x))

### 8.Year End

In [ ]:
def year_end(x):
    """This is used to check if day is end of year"""
    day=calendar.datetime.date.fromisoformat(x).day
    month=calendar.datetime.date.fromisoformat(x).month
    return 1 if (day==31 and month==12) else 0

In [ ]:
train['year_end']=train['date'].apply(lambda x:year_end(x))
test['year_end']=test['date'].apply(lambda x:year_end(x))
final_test['year_end']=final_test['date'].apply(lambda x:year_end(x))

In [ ]:
dates = list(train['date'].unique())

In [ ]:
dates.sort()

In [ ]:
from math import ceil


In [ ]:
ceil((len(dates))*(CV_PART_PRCNT/100))

192

In [ ]:
cv_date = dates[-(ceil((len(dates))*(CV_PART_PRCNT/100))):][0]

In [ ]:

#Cross Validation data will be used for hyperparameter tuning
cv=train[train['date']>=cv_date]
train=train[train['date']<cv_date]

### Time Series Analysis

In [ ]:
#Firstly we will create these Direct features for train  and CV test and final test data
# Code to create one large data for all days
gc.collect()
tt=pd.concat([train,cv,test,final_test])
tt.sort_values(['id','date'],inplace=True)
df=tt.pivot_table(index=['item_id','store_id'],columns='date',values='sales')
df.fillna(0,inplace=True)

1.Rolling mean and Rolling Standard Deviation

In [ ]:
#this code snippet is used to calculate rolling mean and rolling std of sales data with shift of 28 days
# Here we are taking 28 days shift so as to avoid Data Leakage Problem
for aggregate in ['mean','std']:
    for shif in [28]:
        for r in [7,14,30,60,360]:
            roll=df.rolling(r,axis=1).agg(aggregate).shift(shif)
            dates=roll.columns
            name="roll_"+str(r)+"_shift_"+str(shif)+"_"+aggregate
            roll=roll.astype('float16')
            roll.reset_index(level=[0,1],inplace=True)
            roll=pd.melt(roll,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name=name)
            roll.fillna(-1,inplace=True)
            train=train.merge(roll,on=['item_id','store_id','date'])
            cv=cv.merge(roll,on=['item_id','store_id','date'])
            final_test=final_test.merge(roll,on=['item_id','store_id','date'])
            test=test.merge(roll,on=['item_id','store_id','date'])
            print("Feature created named :=",name)
            del roll
            gc.collect()

Feature created named := roll_7_shift_28_mean
Feature created named := roll_14_shift_28_mean
Feature created named := roll_30_shift_28_mean
Feature created named := roll_60_shift_28_mean
Feature created named := roll_360_shift_28_mean
Feature created named := roll_7_shift_28_std
Feature created named := roll_14_shift_28_std
Feature created named := roll_30_shift_28_std
Feature created named := roll_60_shift_28_std
Feature created named := roll_360_shift_28_std


2. Exponential Weighted Average


In [ ]:
# Adding  Exponential weighted average with shift of 28 days
# Shift of 28 days is used to prevent data leakage Problem
roll=df.shift(28,axis=1).ewm(alpha=0.99,axis=1,adjust=False).mean()
dates=roll.columns
roll=roll.astype('float16')
roll.reset_index(level=[0,1],inplace=True)
roll=pd.melt(roll,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name='direct_ewm')
roll.fillna(-1,inplace=True)
train=train.merge(roll,on=['item_id','store_id','date'])
cv=cv.merge(roll,on=['item_id','store_id','date'])
test=test.merge(roll,on=['item_id','store_id','date'])
final_test=final_test.merge(roll,on=['item_id','store_id','date'])
print("Direct Feature created ewa window of size")

Direct Feature created ewa window of size


3. Lag Features

In [ ]:
# Now we will also calculate lag features with lag of 28,35,42,49,56,63,70,77,84,91,96 days
for lag in range(28,100,7):
    i='direct_lag_'+str(lag)
    lag_i=df.shift(lag,axis=1)
    dates=lag_i.columns
    lag_i.reset_index(level=[0,1],inplace=True)
    lag_i=pd.melt(lag_i,id_vars=['item_id','store_id'],value_vars=dates,var_name='date',value_name=i)
    lag_i.fillna(-1,inplace=True)
    lag_i[i]=lag_i[i].astype('int16')
    train=train.merge(lag_i,on=['item_id','store_id','date'])
    cv=cv.merge(lag_i,on=['item_id','store_id','date'])
    test=test.merge(lag_i,on=['item_id','store_id','date'])
    final_test=final_test.merge(lag_i,on=['item_id','store_id','date'])
    print("Feature created for lag",lag)
    del lag_i
    gc.collect()

Feature created for lag 28
Feature created for lag 35
Feature created for lag 42
Feature created for lag 49
Feature created for lag 56
Feature created for lag 63
Feature created for lag 70
Feature created for lag 77
Feature created for lag 84
Feature created for lag 91
Feature created for lag 98


### Saving All features Created

In [ ]:
# train.to_pickle('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.pkl')
train.to_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.pkl',index=False)
cv.to_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/cv1.csv',index=False)
test.to_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/test1.csv',index=False)
final_test.to_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/final_test1.csv',index=False)

In [ ]:
# train.to_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.pkl',index=False)

In [ ]:
# cv.to_pickle('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/cv1.pkl')

In [ ]:
# test.to_pickle('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/test1.pkl')
# final_test.to_pickle('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/final_test1.pkl')

In [ ]:
# tdf = pd.read_csv("/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.csv")
# cdf = pd.read_csv("/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/cv1.csv")

In [ ]:
# tdf = pd.read_csv("/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.csv", chunksize=200)

In [ ]:
# # Initialize an empty DataFrame
# tdf = pd.DataFrame()
# curr=0
# # f"{str(round((curr/40176734), 2))}% completed..."
# # Read the CSV file in chunks
# print("Reading file...")
# for chunk in pd.read_csv('/content/gdrive/MyDrive/PredictiveAnalyticsData/input_data/train1.csv', chunksize=200000):
#     # Process each chunk (e.g., aggregate, filter, etc.)
#     # Append the processed chunk to the main DataFrame
#     tdf = pd.concat([tdf, (chunk)], ignore_index=True)
#     curr+=200000
#     sys.stdout.write(f"\r{str(round((curr/40176734)*100, 2))}% completed...")
#     sys.stdout.flush()

In [ ]:
# tdf.shape()

In [ ]:
# cdf